In [23]:
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
load_dotenv()
import requests
import psycopg2
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)

links_dict = {}

In [ ]:
def create_session_token():
    url = "https://api.voluum.com/auth/access/session"

    payload = {
        "accessId": os.getenv("VOLUUM_ACCESS_ID"),
        "accessKey": os.getenv("VOLUUM_ACCESS_KEY")
    }

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "Accept": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    return response.json()["token"]

access_token = create_session_token()

def get_broadcasts(date):
    url = "https://sms.messagewhiz.com/api/3/Broadcast/List"

    start = 0  # Initialize starting point for pagination
    total_fetched = 0 
    list_of_broadcasts = []
    
    global_check = True
    while True:
        params = {
            'limit': 100,
            'start': start,
        }

        headers = {
            "apikey": "placeholder",
            "Content-Type": "application/json"
        }

        # Send the request
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            break   
        year, month, day = date.split("-")
        broadcast_json = response.json()
        broadcast_res = broadcast_json["result"]
        count = len(broadcast_res)  # Count the number of items fetched in this iteration
        # Process each item
        for item in broadcast_res:
            if date in item["send_date"]:
                list_of_broadcasts.append(item)
            else:
                if f"{year}-{month}-{int(day)+1}" in item["send_date"]:
                    continue
                global_check = False           
                break

        total_fetched += count

        if not global_check:
            break

        if count < 100:  # If less than 100 items, stop fetching
            break

        start += count
    
    return list_of_broadcasts

def get_links():
    url = "https://sms.messagewhiz.com/api/3/Link"

    start = 0  # Initialize starting point for pagination
    total_fetched = 0 
    list_of_links = []
    
    global_check = True
    while True:
        params = {
            'limit': 100,
            'start': start,
        }

        headers = {
            "apikey": "placeholder",
            "Content-Type": "application/json"
        }

        # Send the request
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            break   

        link_json = response.json()
        link_res = link_json["result"]
        count = len(link_res)  # Count the number of items fetched in this iteration
        # Process each item
        for item in link_res:
            list_of_links.append(item)

        total_fetched += count

        if not global_check:
            break

        if count < 100:  # If less than 100 items, stop fetching
            break

        start += count
    
    return list_of_links

def extract_link_id(message):
    return message.split("{{link:")[1].split("}}")[0]

def get_url(link_id):
    return links_dict.get(link_id, None)

def get_campaign_id(link):
    return link.split("m/")[1] if link else None

def format_real_price(price):
    return price/ 100

def get_campaign_data(campaign_id, from_date, to_date):
    url = f"https://panel-api2.voluum.com/report?reportType=table&limit=500&dateRange=yesterday&from={from_date}&to={to_date}&searchMode=TEXT&currency=EUR&sort=visits&direction=ASC&reportDataType=0&offset=0&groupBy=custom-variable-1&groupBy=ip&groupBy=browser-version&groupBy=os-version&column=profit&column=customVariable1&column=visits&column=uniqueVisits&column=suspiciousVisitsPercentage&column=campaignNoConversionsWarning&column=conversions&column=costSources&column=cost&column=revenue&column=roi&column=cv&column=epv&column=cpv&column=errors&column=Click2Reg&column=Reg2FTD&column=customConversions1&column=customRevenue1&column=customConversions2&column=customRevenue2&column=customConversions3&column=customRevenue3&column=osVersion&column=browserVersion&column=actions&column=type&column=clicks&column=suspiciousClicksPercentage&column=suspiciousVisits&column=suspiciousClicks&tz=Etc/GMT&filter1=campaign&filter1Value={campaign_id}&tz=Utc"
    headers = {
        "cwauth-token": access_token
    }
    try:
        response = requests.get(url, headers=headers)
        campaigns = response.json()
    except Exception as e:
        print(f"Error fetching campaign data for campaign ID {campaign_id}: {e}")
        raise e
    return campaigns

def add_broadcast_campaign_data(broadcast_df):
    rows_to_insert = []
    for _, row in tqdm(broadcast_df[broadcast_df["state"] != 2].iterrows(), total=len(broadcast_df[broadcast_df["state"] != 2])):
        # ---- Parse send_date safely ----
        date, time = row["send_date"].split('T')
        from_date = date + 'T' + time.split(':')[0] + ':00:00.000Z'
        # To date will be one hour later
        to_date = date + 'T' + str(int(time.split(':')[0]) + 1).zfill(2) + ':00:00.000Z'

        try: 
            totals = get_campaign_data(
                row["campaign_id"],
                from_date=str(from_date),
                to_date=str(to_date)
            )
            totals = totals["totals"]
        except Exception as e:
            print(f"Error fetching campaign data for campaign ID {row['campaign_id']}: {e}, {totals}")
            raise e
        rows_to_insert.append({
            # Core identifiers
            "id": row["id"],
            "broadcast_id": row["id"],
            "user_id": row["user_id"],
            "department_id": row.get("department", {}).get("id"),
            "department_name": row.get("department", {}).get("name"),

            "campaign_id": row["campaign_id"],
            "link_id": row["link_id"],
            "url": row["url"],

            # Broadcast metadata
            "name": row["name"],
            "type": row["type"],
            "message_body": row["message_body"],
            "enabled": bool(row["enabled"]),
            "trigger_id": row["trigger_id"],
            "parent_id": row["parent_id"],
            "state": row["state"],
            "eb_type": row["eb_type"],
            "company_name": row["company_name"],
            "timezone": row["timezone"],
            "utc_offset": row["utc_offset"],
            "send_now": row["send_now"],

            # Dates
            "send_date": row["send_date"],
            "create_date": pd.to_datetime(row["create_date"], utc=True),
            "created_at": pd.to_datetime(row["created_at"], utc=True),
            "updated_at": pd.to_datetime(row["updated_at"], utc=True),

            "created_by": row["created_by"],
            "updated_by": row["updated_by"],

            # Pricing & counts
            "real_price": row["real_price"],
            "estimated_price": row["estimated_price"],
            "estimated_count": row["estimated_count"],
            "recipient_count": row["recipient_count"],
            "next_segment_size": row["next_segment_size"],

            # DLR stats
            "dlr_sent_count": row["dlr"]["sent_count"] if row.get("dlr") else 0,
            "dlr_delivered_count": row["dlr"]["delivered_count"] if row.get("dlr") else 0,
            "dlr_undelivered_count": row["dlr"]["undelivered_count"] if row.get("dlr") else 0,
            "dlr_rejected_count": row["dlr"]["rejected_count"] if row.get("dlr") else 0,
            "dlr_expired_count": row["dlr"]["expired_count"] if row.get("dlr") else 0,
            "dlr_failed_count": row["dlr"]["failed_count"] if row.get("dlr") else 0,
            "dlr_read_count": row["dlr"]["read_count"] if row.get("dlr") else 0,

            # Broadcast conversion
            "bc_unique_clicks": row["broadcastConversion"]["uniqueClicks"],
            "bc_total_clicks": row["broadcastConversion"]["totalClicks"],
            "bc_recipient_count": row["broadcastConversion"]["recipientCount"],
            "bc_conversion": float(row["broadcastConversion"]["conversion"]),
            "bc_segmented": row["broadcastConversion"]["segmented"],

            # Performance metrics (from totals)
            "click_2_reg": totals["Click2Reg"],
            "reg_2_ftd": totals["Reg2FTD"],
            "clicks": totals["clicks"],
            "conversions": totals["conversions"],
            "cost": totals["cost"],
            "cost_sources": totals["costSources"],
            "cpv": totals["cpv"],

            "custom_conversions_1": totals["customConversions1"],
            "custom_conversions_2": totals["customConversions2"],
            "custom_conversions_3": totals["customConversions3"],

            "custom_revenue_1": totals["customRevenue1"],
            "custom_revenue_2": totals["customRevenue2"],
            "custom_revenue_3": totals["customRevenue3"],

            "cv": totals["cv"],
            "epv": totals["epv"],
            "errors": totals["errors"],
            "profit": totals["profit"],
            "revenue": totals["revenue"],
            "roi": totals["roi"],

            "suspicious_clicks": totals["suspiciousClicks"],
            "suspicious_clicks_percentage": totals["suspiciousClicksPercentage"],
            "suspicious_visits": totals["suspiciousVisits"],
            "suspicious_visits_percentage": totals["suspiciousVisitsPercentage"],

            "unique_visits": totals["uniqueVisits"],
            "visits": totals["visits"],
        })
    return rows_to_insert

In [26]:
date = "2026-01-19"

# Get broadcasts
broadcasts = get_broadcasts(date)
print(f"Number of broadcasts fetched for date: {date} are {len(broadcasts)}")

# Get links
links = get_links()
links_dict = {str(link['id']): link["url"].split("?")[0] for link in links}
print(f"Number of links fetched are {len(links_dict)}")

# Process broadcasts into DataFrame
broadcast_df = pd.DataFrame(broadcasts)
broadcast_df["link_id"] = broadcast_df["message_body"].apply(extract_link_id)
broadcast_df["url"] = broadcast_df["link_id"].apply(get_url)
broadcast_df["campaign_id"] = broadcast_df["url"].apply(get_campaign_id)
broadcast_df["real_price"] = broadcast_df.real_price.apply(format_real_price)

rows_to_insert = add_broadcast_campaign_data(broadcast_df)
print(f"Number of rows to insert: {len(rows_to_insert)}")

Number of broadcasts fetched for date: 2026-01-19 are 13
Number of links fetched are 910


100%|██████████| 13/13 [00:06<00:00,  1.94it/s]

Number of rows to insert: 13


## Tests for Broadcast backup everyday

In [1]:
from app.utils.mmd_data_handler import MMDDataHandler

mmd_handler = MMDDataHandler()

In [2]:
broadcasts = mmd_handler.save_data_for_previous_day()

Fetched 1466 rows.


In [3]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
broadcasts

,id,user_id,name,type,message_body,enabled,trigger_id,send_date,state,parent_id,...,created_at,created_by,updated_by,department_id,department,dlr,broadcastConversion,link_id,url,campaign_id
0,395679800,15696,UK_conv_ong_hlr_2.8K_chunk#15.csv,11,Your account has been credited with GBP500 cash and 50 free spins Use it to boost your night here {{link:6920941}},1,None,2026-01-28T23:59:00.000Z,4,None,...,2026-01-28T23:30:35.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395679800, 'sent_count': 2, 'delivered_count': 30, 'undelivered_count': 0, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0, 'no_balance_count': None}","{'uniqueClicks': 0, 'totalClicks': 0, 'recipientCount': 32, 'conversion': '0.00', 'broadcastId': 395679800, 'segmented': True}",6920941,https://tr.stockholmlab.com/8677e583-87a2-40c8-9a83-9f20e5b91f0a,8677e583-87a2-40c8-9a83-9f20e5b91f0a
1,395679798,15696,UK_conv_ong_hlr_2.8K_chunk#14.csv,11,Your account has been credited with GBP500 cash and 50 free spins Use it to boost your night here {{link:6920941}},1,None,2026-01-28T23:57:00.000Z,4,None,...,2026-01-28T23:30:33.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395679798, 'sent_count': 8, 'delivered_count': 189, 'undelivered_count': 2, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0, 'no_balance_count': None}","{'uniqueClicks': 16, 'totalClicks': 19, 'recipientCount': 199, 'conversion': '8.04', 'broadcastId': 395679798, 'segmented': True}",6920941,https://tr.stockholmlab.com/8677e583-87a2-40c8-9a83-9f20e5b91f0a,8677e583-87a2-40c8-9a83-9f20e5b91f0a
2,395679795,15696,UK_conv_ong_hlr_2.8K_chunk#13.csv,11,Your account has been credited with GBP500 cash and 50 free spins Use it to boost your night here {{link:6920941}},1,None,2026-01-28T23:55:00.000Z,4,None,...,2026-01-28T23:30:31.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395679795, 'sent_count': 7, 'delivered_count': 190, 'undelivered_count': 1, 'rejected_count': 0, 'expired_count': 1, 'failed_count': 0, 'read_count': 0, 'no_balance_count': None}","{'uniqueClicks': 6, 'totalClicks': 6, 'recipientCount': 199, 'conversion': '3.02', 'broadcastId': 395679795, 'segmented': True}",6920941,https://tr.stockholmlab.com/8677e583-87a2-40c8-9a83-9f20e5b91f0a,8677e583-87a2-40c8-9a83-9f20e5b91f0a
3,395679793,15696,UK_conv_ong_hlr_2.8K_chunk#12.csv,11,Your account has been credited with GBP500 cash and 50 free spins Use it to boost your night here {{link:6920941}},1,None,2026-01-28T23:53:00.000Z,4,None,...,2026-01-28T23:30:29.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395679793, 'sent_count': 6, 'delivered_count': 190, 'undelivered_count': 2, 'rejected_count': 0, 'expired_count': 1, 'failed_count': 0, 'read_count': 0, 'no_balance_count': None}","{'uniqueClicks': 12, 'totalClicks': 12, 'recipientCount': 199, 'conversion': '6.03', 'broadcastId': 395679793, 'segmented': True}",6920941,https://tr.stockholmlab.com/8677e583-87a2-40c8-9a83-9f20e5b91f0a,8677e583-87a2-40c8-9a83-9f20e5b91f0a
4,395679791,15696,UK_conv_ong_hlr_2.8K_chunk#11.csv,11,Your account has been credited with GBP500 cash and 50 free spins Use it to boost your night here {{link:6920941}},1,None,2026-01-28T23:51:00.000Z,4,None,...,2026-01-28T23:30:27.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395679791, 'sent_count': 4, 'delivered_count': 193, 'undelivered_count': 2, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0, 'no_balance_count': None}","{'uniqueClicks': 8, 'totalClicks': 8, 'recipientCount': 199, 'conversion': '4.02', 'broadcastId': 395679791, 'segmented': True}",6920941,https://tr.stockholmlab.com/8677e583-87a2-40c8-9a83-9f20e5b91f0a,8677e583-87a2-40c8-9a83-9f20e5b91f0a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,395629714,15696,PT_M

## Tests for cost updates

In [1]:
from app.utils.mmd_data_handler import MMDDataHandler

mmd_handler = MMDDataHandler()

broadcast_df = mmd_handler.update_prices()

Updating costs for date 2026-01-27 to date: 2026-01-28


In [2]:
campaign_ids = broadcast_df["campaign_id"].unique().tolist()
for id in campaign_ids:
    rows = broadcast_df[broadcast_df.campaign_id==id]
    total_cost = rows['real_price'].sum()
    print(id, total_cost)

KeyError: 'campaign_id'

In [1]:
import uuid
from datetime import UTC, timedelta
from app.utils.db_handler import DBHandler
from app.utils.mmd_data_handler import MMDDataHandler
import os
import pandas as pd

mmd_data_handler = MMDDataHandler()
db_handler = DBHandler()

UPLOAD_ROOT = os.getenv("UPLOAD_ROOT", "uploads")
BROADCAST_ROOT = os.path.join(UPLOAD_ROOT, "broadcasts")

FileNotFoundError: [Errno 2] No such file or directory: 'app/utils/country_codes.json'

In [22]:
all_broadcasts = mmd_data_handler.get_broadcasts_until_day("2026-01-01")
broadcasts_df = pd.DataFrame(all_broadcasts)
len(all_broadcasts)

23577

In [ ]:
date_template = "2026-01-31"
previous_date = date_template
broadcast_df = broadcasts_df[broadcasts_df.send_date.str.contains(previous_date)]
broadcast_df["link_id"] = broadcast_df["message_body"].apply(lambda x: x.split("{{link:")[1].split("}}")[0])

broadcast_df["url"] = broadcast_df["link_id"].apply(lambda x: links_dict.get(x, None))
broadcast_df["campaign_id"] = broadcast_df["url"].apply(lambda x: x.split("m/")[1] if x else None)
broadcast_df["real_price"] = broadcast_df["real_price"].apply(lambda price: price / 100)
broadcast_df
total_rows = []
grouped_rows = {}
for _, row in tqdm(broadcast_df.iterrows(), total=len(broadcast_df)):
    if row["state"] in [0, 2]:
        continue
    grouped_rows, processed_row = mmd_data_handler.process_single_row(row, grouped_rows)
    total_rows.append(processed_row)
total_rows_df = pd.DataFrame(total_rows)
file_id = str(uuid.uuid4())
filename = f"MMD_Data_{previous_date}.csv"
raw_file_path = os.path.join("./data", filename)
total_rows_df.to_csv(raw_file_path)
db_handler.create_broadcasts_file_entry(file_id, filename, raw_file_path, f"{previous_date}/T00:00:00.000Z", len(total_rows_df))
db_handler.upsert_broadcast_campaign_stats(total_rows_df)
print(f"Saved for {previous_date}")

/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["link_id"] = broadcast_df["message_body"].apply(lambda x: x.split("{{link:")[1].split("}}")[0])
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["url"] = broadcast_df["link_id"].apply(lambda x: links_dict.get(x, None))
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_4

Saved for 2026-01-10


100%|██████████| 271/271 [00:19<00:00, 14.02it/s]
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["link_id"] = broadcast_df["message_body"].apply(lambda x: x.split("{{link:")[1].split("}}")[0])
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["url"] = broadcast_df["link_id"].apply(lambda x: links_dict.get(x, None))
/var/folde

Saved for 2026-01-11


100%|██████████| 638/638 [00:46<00:00, 13.64it/s]
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["link_id"] = broadcast_df["message_body"].apply(lambda x: x.split("{{link:")[1].split("}}")[0])
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["url"] = broadcast_df["link_id"].apply(lambda x: links_dict.get(x, None))
/var/folde

...

Saved for 2026-01-27


100%|██████████| 1468/1468 [01:20<00:00, 18.13it/s]
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["link_id"] = broadcast_df["message_body"].apply(lambda x: x.split("{{link:")[1].split("}}")[0])
/var/folders/23/7f0y6mrx2d33k4m9pln2t19w0000gn/T/ipykernel_42604/3328687432.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broadcast_df["url"] = broadcast_df["link_id"].apply(lambda x: links_dict.get(x, None))
/var/fol

Saved for 2026-01-28


100%|██████████| 1477/1477 [01:19<00:00, 18.52it/s]


Saved for 2026-01-29


In [ ]:
file_id = str(uuid.uuid4())
filename = f"MMD_Data_{previous_date}.csv"
raw_file_path = os.path.join("./data", filename)
total_rows_df.to_csv(raw_file_path)
db_handler.create_broadcasts_file_entry(file_id, filename, raw_file_path, f"{previous_date}/T00:00:00.000Z", len(total_rows_df))
db_handler.upsert_broadcast_campaign_stats(total_rows_df)